<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Image Web Scraping with Selenium</H1></u></center>

In [1]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests

In [2]:
DRIVER_PATH = './chromedriver_win32/chromedriver'

def image_urls(string, max_links, driver):
    
    def scroll_to_end(driver):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)#sleep_between_interactions    
    
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    driver.get(search_url.format(q=string))

    image_urls = set()
    image_count = 0
    results_start = 0
    
    while image_count < max_links:
        scroll_to_end(driver)

        thumbnail_results = driver.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            
            try:
                img.click()
                time.sleep(1)
            except Exception:
                continue
  
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if image_count >= max_links:
                print(f"Found: {image_count} image links")
                break
        else:
            print("Found:", image_count, "looking for more image links ...")
            time.sleep(30)
            return
            
            load_more_button = driver.find_element_by_css_selector(".mye4qd")
            
            if load_more_button:
                driver.execute_script("document.querySelector('.mye4qd').click();")

        results_start = len(thumbnail_results)

    return image_urls

def save_images(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

In [3]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
search_names = ["bus","cars"]
images_path = f'./img/'

for name in search_names:
    path = os.path.join(images_path,name)
    if not os.path.isdir(path):
        os.mkdir(path)

    driver.get('https://google.com')
    search_box = driver.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(name)
    links = image_urls(name,5,driver)

    for i, link in enumerate(links):
        file_name = f"{i:06}.jpg"
        save_images(path,file_name,link)
driver.quit()

Found: 200 search results. Extracting links from 0:200
Found: 5 image links
SAVED - https://www.stm.info/sites/default/files/styles/stm_article_full/public/ap_tete_serie_new_flyer_4_bandeau_final.jpg?itok=56NVLSjL - AT: ./img/bus\000000.jpg
SAVED - https://shawglobalnews.files.wordpress.com/2019/10/bus-edited.jpg?quality=70&strip=all - AT: ./img/bus\000001.jpg
SAVED - https://images.glaciermedia.ca/polopoly_fs/1.23894071.1563916595!/fileImage/httpImage/image.jpg_gen/derivatives/landscape_804/rapid-bus.jpg - AT: ./img/bus\000002.jpg
SAVED - https://media.wired.com/photos/5cf832279c2a7cd3975976ca/master/pass/Transpo_XcelsiorChargeCharging_TA.jpg - AT: ./img/bus\000003.jpg
SAVED - https://upload.wikimedia.org/wikipedia/commons/6/6a/TransMio_de_Villa_Nueva_GT.jpg - AT: ./img/bus\000004.jpg
Found: 200 search results. Extracting links from 0:200
Found: 5 image links
SAVED - https://specials-images.forbesimg.com/imageserve/5d35eacaf1176b0008974b54/960x0.jpg?cropX1=790&cropX2=5350&cropY1=784&c

## Reference:
- https://selenium-python.readthedocs.io/installation.html